In [1]:
import sys
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.core.framework import graph_pb2
from tensorflow.contrib.slim.python.slim.nets import inception_v3

In [2]:
inc_size = inception_v3.inception_v3.default_image_size
print('inception size: %d' %(inc_size))

inception size: 299


In [3]:
image_path = '/Users/wendyhu/flower_photos/tulips/133858239_3eaa8a91fd_n.jpg'
model_path = '/private/tmp/output_graph.pb'
label_path = '/private/tmp/output_labels.txt'

In [4]:
def load_model():
    graph_def = graph_pb2.GraphDef()
    with tf.gfile.FastGFile(model_path, "rb") as f:
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='illegal_dumping')

In [5]:
def run_inference():
    if not tf.gfile.Exists(image_path):
        tf.logging.fatal('File does not exist %s', image_path)
        return None

    image_data = tf.gfile.FastGFile(image_path, 'rb').read()
    load_model()
    
    with tf.Session() as sess:
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
        predictions = np.squeeze(predictions) 
        top_3 = predictions.argsort()[-3:][::-1]
        
        f = open(label_path, 'rb')
        lines = f.readlines()
        labels = [str(w).replace("\n", "") for w in lines]
        
        for node_id in top_3:
            human_string = labels[node_id]
            score = predictions[node_id]
            print('%s (score = %.5f)' % (human_string, score))
        
        return labels[top_3[0]]

run_inference()

KeyError: "The name 'final_result:0' refers to a Tensor which does not exist. The operation, 'final_result', does not exist in the graph."

Credit: 
https://github.com/eldor4do/TensorFlow-Examples/blob/master/retraining-example.py